# Regularizacija

Do sada smo diskutovali problem loše uslovljenih matrica i njihovu numeričku nestabilnost. U ovoj svesci upoznaćemo neke tehnike koje mogu da poprave korišćenje ovakvih matrica. Razmotrićemo primer rešavanja preuslovljenih sistema i grebene linearne regresije.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Tihonovljeva regularizacija

Posmatrajmo preodređeni sistem $Ax = b$. Ukoliko je matrica $A^TA$ loše uslovljena kod problema najmanjih kvadrata umesto rešavanja osnovnog problema $$min_{x} \|A\cdot x - b\|^2$$ može se rešavati problem $$min_{x} \|A\cdot x - b\|^2 + \alpha ||x||^2$$. Parametar $\alpha$ se zove regularizacioni parametar, a samo rešenje $x$ ima formu $x=(A^{T}A + \alpha I)^{-1}A^{T}b$. 

### Rešavanje loše uslovljenih sistema

Kao poznatu klasu loše uslovljenih matrica naveli smo Hilbertove matrice. Podsetimo se, Hilbertova matrica reda $n$ je matrica čiji su elementi oblika $H_{i, j} = \frac{1}{i+j-1}$ za i, j = 1, 2, 3, ...  Na primer, Hilbertova matrice reda tri je $
\begin{bmatrix}
1 & \frac{1}{2} & \frac{1}{3} \\
\frac{1}{2} & \frac{1}{3} & \frac{1}{4} \\
\frac{1}{3} & \frac{1}{4} & \frac{1}{5} \\
\end{bmatrix}
$.

Uveli smo i funkciju koja kreira Hilbertovu matricu reda $n$.

In [2]:
def hilbert_matrix(n):
    H = np.zeros((n, n))

    for i in range(0, n):
        for j in range(0, n):
            H[i, j] = 1 / ((i + 1) + (j + 1) - 1)
            
    return H

Uvedimo dalje vektor $b$ takav da je vektor jedinica $x$ rešenje sistema $Hx=b$. Elementi vektora $b$ su određeni jednačinom $$b_i = \sum_{j=1}^{j=n}{\frac{1}{i+j-1}}$$, a da bismo mogli da pratimo ponašanja sistema $Hx=b$ posmatraćemo i varijante vektora $b$ određene sa $$b_i = \sum_{j=1}^{j=n}{\frac{1}{i+j-1}} + \sigma R(i)$$ u kojima parametar $\sigma$ predstavlja neku malu vrednost, a $R(i)$ slučajan broj iz intervala $[-1, 1]$.

Za kreiranje takvih vektora koristićemo funkciju `b_values`.

In [3]:
def b_values(n, sigma):
    b = np.zeros(n)
    
    for i in range(0, n):
        b[i] = np.sum ([ 1/ ( (i+1) + (j+1) -1) for j in range(0, n)]) + sigma * np.random.uniform(-1, 1)
        
    return b

Za rešavanje sistema $Hx=b$ možemo koristiti funkciju `solve_hilbert_system`:

In [4]:
def solve_hilbert_system(n, sigma):
    H = hilbert_matrix(n)
    b = b_values(n, sigma)
    
    return np.linalg.inv(H).dot(b)

Za rešavanje regularizovane varijante sistema možemo koristiti funkciju `solve_regularized_hilbert_system`. Neka podrazumevana vrednost parametra regularizacije bude $0.5$. Njegovo optimalno određivanje ćemo diskutovati u nastavku.

In [ ]:
# TODO

U slučaju kada je $\sigma=10^{-5}$ za Hilbertovu matricu reda deset, funkcije daju prilično različita rešenja. Podsetimo se, vektor jedinica je tačno rešenje sistema $Hx=b$.

In [ ]:
# TODO

Možemo da prikažemo i kako se ponaša norma razlike tačnog i regularizovanog rešenja za različite vrednosti regularizacionog parametra $\alpha$.

In [ ]:
# TODO

### Grebena regresija

U kontekstu linearne regresije pri rešavanju sistema $A\beta = Y$ metodom najmanjih kvadrata, gde se minimizuje vrednost izraza $||Y - \beta A||_2^2$ i čije je rešenje dato u obliku $\beta=(A^{T}A)^{-1}A^{T}Y$ može se posmatrati regularizovana forma kojom se vrši minimizacija $||Y - \beta A||_2^2 + \alpha ||\beta||_2^2$ za pogodno izabran hiperparametar $\alpha$. Umesto linearne, tada se govori o grebenoj regresiji, a izraz $\alpha ||\beta||_2^2$ se naziva regularizacioni element.

Dalje ćemo na osnovu podataka u datoteci *advertising.csv* koja sumira statistike u vezi sa reklamiranjima i ostvarenom zaradom, kreirati grebeni model koji može da predvidi zaradu. Demonstriraćemo i tehniku za određivanje optimalne vrednosti hiperparametara iz zadatog skupa koja koristi validacioni skup.

In [19]:
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing

Najpre učitajmo podatke i izvucimo osnovne informacije o njima.

Matricu nezavisnih promenljivih $X$ i vektor zavisnih promenljivih $y$ ćemo podeliti na skup za treniranje, skup za validaciju i skup za testiranje. Ideja je da na skupu za testiranje testiramo performanse naših modela, da validacioni skup iskoristimo za ocenu optimalne vrednosti regularizacionog parametra, a da skup za treniranje iskoristimo za samo treniranje modela. Razmera podele ovih skupova će biti 60%: 20%: 20%. Ovakva podela skup će pomoći da se izbegne problem preprilagođavanja (engl. *overfitting*) modela podacima.

<img src='assets/train_validation_test.png'>

Za kreiranje ovih skupova možemo iskoristiti funkciju `train_test_split`. 

Prilikom obučavanja modela linearne regresije može se desiti da su numeričke vrednosti podataka neuravnotežene, na primer, da je za neke atribute maksimalna vrednosti nekoliko stotina, a za druge nekoliko jedinica. Da bismo mogli lakše da interpretiramo tako dobijene rezultate, podatke `standardizujemo` tako što ih transformišemo (po kolonama) tako da njihova srednja vrednost bude 0, a standardna devijacija 1. Osim ove svrhe, standardizacija podataka utiče na stabilnost i brzinu treniranja modela pa ju je poželjno uvek primenjivati. Za samu standardizaciju koristićemo klasu `StandardScaler` biblioteke `sklearn`. 

In [35]:
# TODO

Sada možemo da odredimo model linearne regresije, a probaćemo i sa dva modela grebene regresije sa, na primer, parametrima $\alpha = 0.5$ i $\alpha = 10^{10}$. U sva tri slučaja, izračunaćemo i srednjekvadratnu grešku. Za grebenu regresiju se na nivou biblioteke koristi klasa `Ridge`. Način korišćenja je sličan načinu korišćenja klase `LinearRegression` uz dodatak vrednosti parametra $\alpha$.

In [ ]:
# TODO